In [1]:
!pip install apyori
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from apyori import apriori
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_validate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=5f3047357735c68c8d95e94416a151dc384936355ca8ae2a9bde55277b3492fa
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [2]:
df = pd.read_csv('heart_2020_cleaned.csv')


df.head()
        


# df.loc[df['AgeCategory'].str.contains("18-24"),'NumAgeCategory'] = 0
# df.loc[df['AgeCategory'].str.contains("25-29"),'NumAgeCategory'] = 1
# df.loc[df['AgeCategory'].str.contains("30-34"),'NumAgeCategory'] = 2

# df.loc[df['AgeCategory'].str.contains("35-39"),'NumAgeCategory'] = 3
# df.loc[df['AgeCategory'].str.contains("40-44"),'NumAgeCategory'] = 4
# df.loc[df['AgeCategory'].str.contains("45-49"),'NumAgeCategory'] = 5
# df.loc[df['AgeCategory'].str.contains("50-54"),'NumAgeCategory'] = 6
# df.loc[df['AgeCategory'].str.contains("55-59"),'NumAgeCategory'] = 7
# df.loc[df['AgeCategory'].str.contains("60-64"),'NumAgeCategory'] = 8
# df.loc[df['AgeCategory'].str.contains("65-69"),'NumAgeCategory'] = 9
# df.loc[df['AgeCategory'].str.contains("70-74"),'NumAgeCategory'] = 10
# df.loc[df['AgeCategory'].str.contains("75-79"),'NumAgeCategory'] = 11
# df.loc[df['AgeCategory'].str.contains("80"),'NumAgeCategory'] = 12

# df.loc[df['Race'].str.contains("American Indian/Alaskan Native"),'NumRace'] = 0
# df.loc[df['Race'].str.contains("Asian"),'NumRace'] = 1
# df.loc[df['Race'].str.contains("Black"),'NumRace'] = 2

# df.loc[df['Race'].str.contains("Hispanic"),'NumRace'] = 3
# df.loc[df['Race'].str.contains("White"),'NumRace'] = 4
# df.loc[df['Race'].str.contains("Other"),'NumRace'] = 5

# df.loc[df['Sex'].str.contains("Female"),'NumSex'] = 0
# df.loc[df['Sex'].str.contains("Male"),'NumSex'] = 1

df.loc[df['HeartDisease'].str.contains("No"),'NumHD'] = 0
df.loc[df['HeartDisease'].str.contains("Yes"),'NumHD'] = 1


df=df.dropna()



df.head()


,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer,NumHD
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes,0.0
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No,0.0
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No,0.0
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes,0.0
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No,0.0


In [3]:
print("total" ,len(df))
df_HD = df.loc[df['NumHD'] == 1]
df_NHD = df.loc[df['NumHD'] == 0]
print('hd', len(df_HD))
print('nhd', len(df_NHD))
#get equal amount of sample of each
df_HD = df_HD.sample(27373)
print('curr total', len(df_HD))
df_NHD = df_NHD.sample(27373)
print('curr total', len(df_NHD))
# merge the two types
df = pd.concat([df_HD, df_NHD], axis=0)
print(len(df))
df = df.drop(columns=['NumHD'])
df.head()

total 319795
hd 27373
nhd 292422
curr total 27373
curr total 27373
54746


,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
20280,Yes,20.12,No,No,Yes,3.0,0.0,No,Female,80 or older,White,No,Yes,Good,12.0,No,No,No
54435,Yes,19.65,Yes,No,No,20.0,0.0,Yes,Female,75-79,White,No,No,Fair,8.0,No,No,Yes
253420,Yes,21.95,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,No,Good,9.0,No,No,No
15844,Yes,31.32,No,No,No,2.0,0.0,Yes,Female,80 or older,White,No,No,Good,9.0,No,No,Yes
188195,Yes,30.11,No,Yes,No,30.0,0.0,Yes,Female,80 or older,White,No,Yes,Fair,6.0,Yes,No,No


In [4]:

    
def aabuilderm(d):
    res = ""
    
    
    
#     switch case wouldnt work
    if (d['AgeCategory'] == "18-24"):
        res = res + ("18-24")
    
    elif (d['AgeCategory'] == "25-29"):
        res = res + ("25-29")
    elif (d['AgeCategory'] == "30-34"):
        res = res + ("30-34")
    elif (d['AgeCategory'] == "35-39"):
        res = res + ("35-39")
    elif (d['AgeCategory'] == "40-44"):
        res = res + ("40-44")
    elif (d['AgeCategory'] == "45-49"):
        res = res + ("45-49")
    elif (d['AgeCategory'] == "50-54"):
        res = res + ("50-54")
    elif (d['AgeCategory'] == "55-59"):
        res = res + ("55-59")
    elif (d['AgeCategory'] == "60-64"):
        res = res + ("60-64")
    elif (d['AgeCategory'] == "65-69"):
        res = res + ("65-69")
    elif (d['AgeCategory'] == "70-74"):
        res = res + ("70-74")
    elif (d['AgeCategory'] == "75-79"):
        res = res + ("75-79")
    elif (d['AgeCategory'] == "80 or older"):
        res = res + ("80 or older")
        
        
    if (d['Race'] == "American Indian/Alaskan Native"):
        res = res + (", American Indian/Alaskan Native")
    
    elif (d['Race'] == "Asian"):
        res = res + (", Asian")
    elif (d['Race'] == "Black"):
        res = res + (", Black")
    elif (d['Race'] == "Hispanic"):
        res = res + (", Hispanic")
    elif (d['Race'] == "White"):
        res = res + (", White")
    elif (d['Race'] == "Other"):
        res = res + (", Other race")
        
    if (d['Sex'] == "Female"):
        res = res + (", Female")
    
    elif (d['Sex'] == "Male"):
        res = res + (", Male")
    
    if (d['BMI'] < 18.5):
        res = res + (", Underweight")
    elif (d['BMI'] < 24.9):
        res = res + (", In Healthy Weight Range")
    elif (d['BMI'] < 29.9):
        res = res + (", Overweight")
    else:
        res = res + (", Obese")
        
    if (d['PhysicalHealth'] == 0):
        res = res + (", No days bad Physical Health")
    elif (d['PhysicalHealth'] < 10):
        res = res + (", Less than 10 days bad Physical Health")
    elif (d['PhysicalHealth'] < 20):
        res = res + (", Less than 20 days bad Physical Health")
    else:
        res = res + (", 20+ days bad Physical Health")
    
    
    if (d['MentalHealth'] == 0):
        res = res + (", No days bad Mental Health")
    elif (d['MentalHealth'] < 10):
        res = res + (", Less than 10 days bad Mental Health")
    elif (d['MentalHealth'] < 20):
        res = res + (", Less than 20 days bad Mental Health")
    else:
        res = res + (", 20+ days bad Mental Health")
        
        
        
    if (d['DiffWalking'] == "No"):
        res = res + (", No Difficulty Walking")
    
    elif (d['DiffWalking'] == "Yes"):
        res = res + (", Yes Difficulty Walking")
    
        
        
    if (d['AlcoholDrinking'] == "No"):
        res = res + (", No Alchohol Drinking")
    
    elif (d['AlcoholDrinking'] == "Yes"):
        res = res + (", Yes Alchohol Drinking")
        
    
    if (d['Stroke'] == "No"):
        res = res + (", No Stroke")
    
    elif (d['Stroke'] == "Yes"):
        res = res + (", Yes Stroke")
        
        
    if (d['Diabetic'] == "No"):
        res = res + (", No Diabetic")
    
    else:# Helps keep the data more balanced
        res = res + (", Yes Diabetic or Borderline Diabetes")
        
        
    if (d['PhysicalActivity'] == "No"):
        res = res + (", No Physical Activity")
    
    elif (d['PhysicalActivity'] == "Yes"):
        res = res + (", Yes Physical Activity")
        
    if (d['GenHealth'] == "Excellent"):
        res = res + (", Excellent General Health")
    
    elif (d['GenHealth'] == "Fair"):
        res = res + (", Fair General Health")
    elif (d['GenHealth'] == "Good"):
        res = res + (", Good General Health")
    elif (d['GenHealth'] == "Poor"):
        res = res + (", Poor General Health")
    elif (d['GenHealth'] == "Very Good"):
        res = res + (", Very Good General Health")
        
    
    if (d['SleepTime'] < 8):
        res = res + (", Less than 8 hours of Sleep")
    elif (d['SleepTime'] < 10):
        res = res + (", Less than 10 hours of Sleep")
    else:
        res = res + (", 10+ hours of Sleep")
        
    if (d['KidneyDisease'] == "No"):
        res = res + (", No Kidney Disease")
    
    elif (d['KidneyDisease'] == "Yes"):
        res = res + (", Yes Kidney Disease")
        
    if (d['SkinCancer'] == "No"):
        res = res + (", No Skin Cancer")
    
    elif (d['SkinCancer'] == "Yes"):
        res = res + (", Yes Skin Cancer")
        
    if (d['Asthma'] == "No"):
        res = res + (", No Asthma")
    
    elif (d['Asthma'] == "Yes"):
        res = res + (", Yes Asthma")
        
    if (d['Smoking'] == "No"):
        res = res + (", No Smoking")
    
    elif (d['Smoking'] == "Yes"):
        res = res + (", Yes Smoking")
    
        
    

        
        
        
    
        
    if (d['HeartDisease'] == "No"):
        res = res + (", No Heart Disease")
    
    elif (d['HeartDisease'] == "Yes"):
        res = res + (", Yes Heart Disease")
    
    
    return res
    
df['categories'] = df.apply(aabuilderm, axis=1)
print(df)
df.info()

       HeartDisease    BMI Smoking AlcoholDrinking Stroke  PhysicalHealth  \
20280           Yes  20.12      No              No    Yes             3.0   
54435           Yes  19.65     Yes              No     No            20.0   
253420          Yes  21.95      No              No    Yes             0.0   
15844           Yes  31.32      No              No     No             2.0   
188195          Yes  30.11      No             Yes     No            30.0   
...             ...    ...     ...             ...    ...             ...   
75693            No  27.32      No              No     No             4.0   
185510           No  26.26      No              No     No             0.0   
16629            No  25.15     Yes              No     No            20.0   
21121            No  20.37      No              No     No             0.0   
191293           No  33.00     Yes              No     No             0.0   

        MentalHealth DiffWalking     Sex  AgeCategory   Race  \
20280      

In [ ]:
keywords = []

for l in df['categories']:
    keywords_list = l.split(',')
    stripped_list = [s.strip() for s in keywords_list]
    if len(keywords_list) > 1:
        keywords.append(stripped_list)

rules = list(apriori(keywords, min_support = 0.1, min_confidence = .70, min_lift = 1)) 
# played with confidence, not much changes with even lower confidence values

# print(rules)
dfrr = pd.DataFrame(columns=('items', 'support', 'ordered_statistics', 'items_base', 'items_add', 'confidence', 'lift'))
items = []
support = []
ordered_statistics = []
items_add = []
items_base = []
confidence = []
lift = []

for rule in rules:
    for stat in rule.ordered_statistics:
        items.append(rule.items)
        support.append(rule.support)
        ordered_statistics.append(rule.ordered_statistics)
        items_add.append(stat.items_add)
        items_base.append(stat.items_base)
        confidence.append(stat.confidence)
        lift.append(stat.lift)
        
dfrr['items'] = list(map(set,items))
dfrr['support'] = support
dfrr['ordered_statistics'] = list(map(set, ordered_statistics))
dfrr['items_add'] = list(map(set, items_add))
dfrr['items_base'] = list(map(set, items_base))
dfrr['confidence'] = confidence
dfrr['lift'] = lift        


dfrr['hashd'] = dfrr.apply(lambda row: row.astype(str).str.contains('yes Heart Disease').any(), axis=1)

# excluding non-hd related rules
dfrr = dfrr[dfrr.hashd != False]
mdf = dfrr.sort_values(['hashd', 'lift'], ascending = [False, False], inplace = True)

print("All the rules with at least 0.1 support and 0.7 confidence relevant to having Heart Disease\n")
print(dfrr)

print("\n\n")

print("The rules with the highest lift: \n")
first_row_dfrr = dfrr.iloc[:3]
print(first_row_dfrr)



In [ ]:
keywords = []

for l in df['categories']:
    keywords_list = l.split(',')
    stripped_list = [s.strip() for s in keywords_list]
    if len(keywords_list) > 1:
        keywords.append(stripped_list)

rules = list(apriori(keywords, min_support = 0.1, min_confidence = 0.7, min_lift = 1))


# print(rules)
dfrr = pd.DataFrame(columns=('items', 'support', 'ordered_statistics', 'items_base', 'items_add', 'confidence', 'lift'))
items = []
support = []
ordered_statistics = []
items_add = []
items_base = []
confidence = []
lift = []

for rule in rules:
    for stat in rule.ordered_statistics:
        items.append(rule.items)
        support.append(rule.support)
        ordered_statistics.append(rule.ordered_statistics)
        items_add.append(stat.items_add)
        items_base.append(stat.items_base)
        confidence.append(stat.confidence)
        lift.append(stat.lift)
        
dfrr['items'] = list(map(set,items))
dfrr['support'] = support
dfrr['ordered_statistics'] = list(map(set, ordered_statistics))
dfrr['items_add'] = list(map(set, items_add))
dfrr['items_base'] = list(map(set, items_base))
dfrr['confidence'] = confidence
dfrr['lift'] = lift        


dfrr['nohd'] = dfrr.apply(lambda row: row.astype(str).str.contains('no Heart Disease').any(), axis=1)

# excluding non-hd  rules
dfrr = dfrr[dfrr.nohd != False]
mdf = dfrr.sort_values(['nohd', 'lift'], ascending = [False, False], inplace = True)

print("All the rules with at least 0.1 support and 0.7 confidence relevant to having Heart Disease\n")
print(dfrr)

print("\n\n")

print("The rules with the highest lift: \n")
first_row_dfrr = dfrr.iloc[:3]
print(first_row_dfrr)

In [ ]:
keywords = []

for l in df['categories']:
    keywords_list = l.split(',')
    stripped_list = [s.strip() for s in keywords_list]
    if len(keywords_list) > 1:
        keywords.append(stripped_list)

rules = list(apriori(keywords, min_support = 0.05, min_confidence = .70, min_lift = 1)) 
# played with confidence, not much changes with even lower confidence values

# print(rules)
dfrr = pd.DataFrame(columns=('items', 'support', 'ordered_statistics', 'items_base', 'items_add', 'confidence', 'lift'))
items = []
support = []
ordered_statistics = []
items_add = []
items_base = []
confidence = []
lift = []

for rule in rules:
    for stat in rule.ordered_statistics:
        items.append(rule.items)
        support.append(rule.support)
        ordered_statistics.append(rule.ordered_statistics)
        items_add.append(stat.items_add)
        items_base.append(stat.items_base)
        confidence.append(stat.confidence)
        lift.append(stat.lift)
        
dfrr['items'] = list(map(set,items))
dfrr['support'] = support
dfrr['ordered_statistics'] = list(map(set, ordered_statistics))
dfrr['items_add'] = list(map(set, items_add))
dfrr['items_base'] = list(map(set, items_base))
dfrr['confidence'] = confidence
dfrr['lift'] = lift        


dfrr['hashd'] = dfrr.apply(lambda row: row.astype(str).str.contains('yes Heart Disease').any(), axis=1)

# excluding non-hd related rules
dfrr = dfrr[dfrr.hashd != False]
mdf = dfrr.sort_values(['hashd', 'lift'], ascending = [False, False], inplace = True)

print("All the rules with at least 0.15 support and 0.7 confidence relevant to having Heart Disease\n")
print(dfrr)

print("\n\n")

print("The rules with the highest lift: \n")
first_row_dfrr = dfrr.iloc[:3]
print(first_row_dfrr)



In [ ]:
keywords = []

for l in df['categories']:
    keywords_list = l.split(',')
    stripped_list = [s.strip() for s in keywords_list]
    if len(keywords_list) > 1:
        keywords.append(stripped_list)

rules = list(apriori(keywords, min_support = 0.05, min_confidence = 0.7, min_lift = 1))


# print(rules)
dfrr = pd.DataFrame(columns=('items', 'support', 'ordered_statistics', 'items_base', 'items_add', 'confidence', 'lift'))
items = []
support = []
ordered_statistics = []
items_add = []
items_base = []
confidence = []
lift = []

for rule in rules:
    for stat in rule.ordered_statistics:
        items.append(rule.items)
        support.append(rule.support)
        ordered_statistics.append(rule.ordered_statistics)
        items_add.append(stat.items_add)
        items_base.append(stat.items_base)
        confidence.append(stat.confidence)
        lift.append(stat.lift)
        
dfrr['items'] = list(map(set,items))
dfrr['support'] = support
dfrr['ordered_statistics'] = list(map(set, ordered_statistics))
dfrr['items_add'] = list(map(set, items_add))
dfrr['items_base'] = list(map(set, items_base))
dfrr['confidence'] = confidence
dfrr['lift'] = lift        


dfrr['nohd'] = dfrr.apply(lambda row: row.astype(str).str.contains('no Heart Disease').any(), axis=1)

# excluding non-hd  rules
dfrr = dfrr[dfrr.nohd != False]
mdf = dfrr.sort_values(['nohd', 'lift'], ascending = [False, False], inplace = True)

print("All the rules with at least 0.15 support and 0.7 confidence relevant to having Heart Disease\n")
print(dfrr)

print("\n\n")

print("The rules with the highest lift: \n")
first_row_dfrr = dfrr.iloc[:3]
print(first_row_dfrr)